In [6]:
import numpy as np
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(20,20))
plt.rc('axes', labelsize=24)
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.rc('legend', fontsize=22)

Largo = 10 #En centimetros
Temp_inicial = 0
Temp_extremos = [100, 50]
k = 0.835 #cm^2/s
tiempo_limite = 5000 #segundo


deltax = 2 #cm
deltat = 1 #s
frec_muestreo = 10 # 50 #cada cuantos deltat tomo datos.

In [25]:
def metodo_explicito (Largo,temp_inicial,temp_extremos,k,timepo_limite,deltax,deltat,frec_muestreo):

    #Parametros inciales
    Frec_muestreo = frec_muestreo*deltat
    puntos_pos = np.arange(0,Largo+deltax,deltax) #distancias en el x a la cuales resuelvo
    puntos_tiempo = np.arange(0,tiempo_limite+deltat,deltat) #tiempos a los que resuelvo
    puntos_frec = np.arange(0,tiempo_limite+frec_muestreo,frec_muestreo) #tiempos en los que tomo datos
    Coeficiente = (k*deltat)/(deltax**2)

    #cargo el primer momento de la barra 
    T = np.ones(len(puntos_pos))*Temp_inicial 
    T[0] = Temp_extremos[0] ; T[-1] = Temp_extremos[1] 

    #Necesito un vector que vaya almacenando los datos del siguiente punto
    T_almacenar = []
    Temp_sig = T.copy()
    
    for tiempo in puntos_tiempo:
      l = 1
      for posicion in puntos_pos:   
        if posicion == 0:
          Temp_sig[0] = Temp_extremos[0]
        elif posicion == Largo:
          Temp_sig[-1] = Temp_extremos[1]
        else:
          Temp_sig[l] = T[l]+ ((Coeficiente)*(T[l+1]-2*T[l]+T[l-1]))
          l += 1

      if tiempo in puntos_frec:
          T_almacenar.append(T)
      elif tiempo == tiempo_limite: 
          T_almacenar.append(Temp_sig)
            #asi siempre guarda el ultimo tiempo salvo salga por el error aunque 
            #creo que puede ser redundante solo que con algunos tiempos que probe a veces 
            #lo tomaba a veces no (antes de agregar la salida por error)

      
      T = Temp_sig.copy()
      error = np.dot((T-T_almacenar[-1]),(T-T_almacenar[-1]))
      if error < 1e-7:
        break
      #mdf_comment también conviene poner u break en el caso que diverge:
      if any( T_almacenar[-1] > 1000 ):
        break
    return T_almacenar, puntos_pos


In [26]:
T_almacenar,puntos_pos = metodo_explicito (
    Largo,Temp_inicial,Temp_extremos,k,tiempo_limite,deltax,3,frec_muestreo)

%matplotlib auto

import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='jshtml')

print(len(T_almacenar))
fig1 = plt.figure()

def animacion1 (i):
    plt.plot(puntos_pos,T_almacenar[i])

grafica = animation.FuncAnimation(fig1,animacion1,range(len(T_almacenar)))
grafica



Using matplotlib backend: Qt5Agg
4


In [4]:
def metodo_implicito (Largo,temp_inicial,temp_extremos,k,timepo_limite,deltax,deltat,frec_muestreo):

    #Parametros inciales
    Frec_muestreo = frec_muestreo*deltat
    puntos_pos = np.arange(0,Largo+deltax,deltax)
    puntos_tiempo = np.arange(0,tiempo_limite+deltat,deltat)
    puntos_frec = np.arange(0,tiempo_limite+frec_muestreo,frec_muestreo)
    Coeficiente = (k*deltat)/(deltax**2)
   

    #genero la matriz que debo resolver
    Matriz = np.eye(len(puntos_pos))
    for i in range(1,len(puntos_pos)-1):
       for j in range(1,len(puntos_pos)-1):
          if i == j:
             Matriz[j,i] = 1 + 2 * Coeficiente
             Matriz[j,i-1] = -Coeficiente
             Matriz[j,i+1] = -Coeficiente


    #cargo el primer momento de la barra 
    T = np.ones(len(puntos_pos))*Temp_inicial 
    T[0] = Temp_extremos[0] ; T[-1] = Temp_extremos[1] 

    #Necesito un vector que vaya almacenando los datos del siguiente punto
    T_almacenar = [T]
    Vector_B = T.copy()


    for tiempo in puntos_tiempo:
        Temp_sig = np.linalg.solve(Matriz,T)

        if tiempo in puntos_frec:
            T_almacenar.append(Temp_sig)
        elif tiempo == tiempo_limite:
            T_almacenar.append(Temp_sig)

        T = Temp_sig.copy()
        l = 1
        for posicion in puntos_pos: 
            if posicion == 0:
               Temp_sig[0] = Temp_extremos[0]
            elif posicion == Largo:
               Temp_sig[-1] = Temp_extremos[1]
            else:
               Temp_sig[l] = T[l]+ ((Coeficiente)*(T[l+1]-2*T[l]+T[l-1]))
               l += 1
        error = np.dot((T-T_almacenar[-1]),(T-T_almacenar[-1]))
        if error < 1e-7:
          break

    return T_almacenar, puntos_pos

In [18]:
T_almacenar2,puntos_pos2 = metodo_implicito (
    Largo,
    Temp_inicial,
    Temp_extremos,
    k,tiempo_limite,deltax,0.5,
    frec_muestreo)


%matplotlib auto

import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='jshtml')

#MDF_COMMENT print(len(T_almacenar))
#MDF_COMMENT ojo, aca tiene que ser almacenar2!
print(len(T_almacenar2))

fig2 = plt.figure()
def animacion2 (i):
    plt.plot(puntos_pos2,T_almacenar2[i])



#MDF_COMMENT ojo, aca tiene que ser almacenar2!
grafica2 = animation.FuncAnimation(fig2,animacion2,range(len(T_almacenar2)))


grafica2


Using matplotlib backend: Qt5Agg
11


In [16]:
np.shape(T_almacenar2)

(13, 6)

In [17]:
len(T_almacenar2)

13